In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import load_model

from utils.custom_loss import mae_wrap_angle, mse_wrap_angle, angle_diff_deg
from utils.load_data_raw import DataGenerator_raw, load_raw_all_h5
from utils.dataset_split import *
from utils.eval import *
from utils.utils import get_filelist, open_json

NUM_WORKER = 4

Using TensorFlow backend.


In [3]:
# load filelist
model_dir = '../data/models_trained/'
filelist = get_filelist(model_dir)

# load data
dset_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/raw_nf10_mid/database_raw_nf10_scaledMM.h5'
feat, targ, ID_ref, pos_t, cond_t, par = load_raw_all_h5(dset_dir)

# load unscaled data
dset_nsc_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/raw_nf10_mid/database_raw_nf10.h5'
feat_nsc, targ_nsc, _, _, _, _ = load_raw_all_h5(dset_nsc_dir)

# create parameter dicts for the test batch generators
params = create_test_params(feat, targ, par, shuffle=False)
params_nsc = create_test_params(feat_nsc, targ_nsc, par, shuffle=False)

part = {}
for i in range(len(cond_t)):
    cond = cond_t.iloc[i]
    cond_ids_test, pos_ids_test, subject_ids_test = get_subset_ids([cond[0]], cond_t)
    part[cond[0]] = get_subset_sample_idx(ID_ref, cond_ids_test, pos_ids_test, subject_ids_test)

In [4]:
h_flist = [x for x in filelist if 'history' in x]
pt_flist = [x for x in filelist if 'partition_test' in x]
m_flist = [x for x in filelist if x not in h_flist+pt_flist]

for i in range(len(m_flist)):
    s = ' -' if i<10 else '-'
    print(i, s, m_flist[i])
    print(i, s, h_flist[i])
    print(i, s, pt_flist[i])

0  - m032-16_maew_adam_bs1024_t-NR006-NM027_toptest.h5
0  - m032-16_maew_adam_bs1024_t-NR006-NM027_toptest_history.json
0  - m032-16_maew_adam_bs1024_t-NR006-NM027_toptest_partition_test.json
1  - m032-16_maew_adam_bs1024_t-NR006_toptest.h5
1  - m032-16_maew_adam_bs1024_t-NR006_toptest_history.json
1  - m032-16_maew_adam_bs1024_t-NR006_toptest_partition_test.json
2  - m064_mae_adam_bs1024_t-split_wraptest.h5
2  - m064_mae_adam_bs1024_t-split_wraptest_history.json
2  - m064_mae_adam_bs1024_t-split_wraptest_partition_test.json
3  - m064_maew_adam_bs1024_t-split_wraptest.h5
3  - m064_maew_adam_bs1024_t-split_wraptest_history.json
3  - m064_maew_adam_bs1024_t-split_wraptest_partition_test.json
4  - m128-128-64_maew_adam_bs1024_t-NR006_toptest.h5
4  - m128-128-64_maew_adam_bs1024_t-NR006_toptest_history.json
4  - m128-128-64_maew_adam_bs1024_t-NR006_toptest_partition_test.json
5  - m128_mae_adam_bs1024_t-split_wraptest.h5
5  - m128_mae_adam_bs1024_t-split_wraptest_history.json
5  - m128_mae

In [5]:
# load models, train history and test split IDs (partition['test'])

#wraptest
m64_mae = load_model(model_dir + m_flist[2], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h64_mae  = open_json(model_dir, h_flist[2])
pt64_mae = open_json(model_dir, pt_flist[2])

m64_maew = load_model(model_dir + m_flist[3], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h64_maew = open_json(model_dir, h_flist[3])
pt64_maew = open_json(model_dir, pt_flist[3])

m128_mae = load_model(model_dir + m_flist[5], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128_mae  = open_json(model_dir, h_flist[5])
pt128_mae = open_json(model_dir, pt_flist[5])

m128_maew = load_model(model_dir + m_flist[10], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128_maew = open_json(model_dir, h_flist[10])
pt128_maew = open_json(model_dir, pt_flist[10])

m256_mae = load_model(model_dir + m_flist[14], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h256_mae  = open_json(model_dir, h_flist[14])
pt256_mae = open_json(model_dir, pt_flist[14])

m256_maew = load_model(model_dir + m_flist[15], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h256_maew = open_json(model_dir, h_flist[15])
pt256_maew = open_json(model_dir, pt_flist[15])                      

#normtest
m128_maew_nn = load_model(model_dir + m_flist[9], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128_maew_nn = open_json(model_dir, h_flist[9])
pt128_maew_nn = open_json(model_dir, pt_flist[9])

#toptest
m128_tt = load_model(model_dir + m_flist[6], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128_tt = open_json(model_dir, h_flist[6])
pt128_tt = open_json(model_dir, pt_flist[6])

m128bn_bs2048_tt = load_model(model_dir + m_flist[13], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128bn_bs2048_tt = open_json(model_dir, h_flist[13])
pt128bn_bs2048_tt = open_json(model_dir, pt_flist[13])

m128bn_bs256_tt = load_model(model_dir + m_flist[11], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128bn_bs256_tt = open_json(model_dir, h_flist[11])
pt128bn_bs256_tt = open_json(model_dir, pt_flist[11])

m128_128_64_tt = load_model(model_dir + m_flist[4], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h128_128_64_tt = open_json(model_dir, h_flist[4])
pt128_128_64_tt = open_json(model_dir, pt_flist[4])

m512_256_tt = load_model(model_dir + m_flist[16], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h512_256_tt = open_json(model_dir, h_flist[16])
pt512_256_tt = open_json(model_dir, pt_flist[16])

m32_16_tt = load_model(model_dir + m_flist[0], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
h32_16_tt  = open_json(model_dir, h_flist[0])
pt32_16_tt = open_json(model_dir, pt_flist[0])

W1022 20:13:12.413083 139662188214080 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1022 20:13:12.434354 139662188214080 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1022 20:13:12.532016 139662188214080 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1022 20:13:12.533201 139662188214080 deprecation_wrapper.py:119] From /home/feliximmohr/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.

## Test: normalized vs. not normalized

    Model 1: 128-128-1    m128_maew_wraptest   vs.  m128_maew_normtest_neg


In [ ]:
# Model 1 normalized: m128_maew
tbg128_maew = model_complete_eval(m128_maew, h128_maew, pt128_maew, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_maew, tbg128_maew, b_idx=0)

In [ ]:
# Model 1 not normalized: m128_maew_nn
tbg128_maew_nn = model_complete_eval(m128_maew_nn, h128_maew_nn, pt128_maew_nn, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_maew_nn, tbg128_maew_nn, b_idx=0)

## Test: custom vs. non-custom loss function

    Model 1: 64-32-1      m64_maew_wraptest   vs.  m64_mae_wraptest
    Model 2: 128-128-1    m128_maew_wraptest  vs.  m128_mae_wraptest
    Model 3: 256-128-1    m256_maew_wraptest  vs.  m256_mae_wraptest


In [ ]:
# Model 1 custom loss: m64_maew
tbg64_maew = model_complete_eval(m64_maew, h64_maew, pt64_maew, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m64_maew, tbg64_maew, b_idx=0)

In [ ]:
# Model 1 mae loss: m64_mae
tbg64_mae = model_complete_eval(m64_mae, h64_mae, pt64_mae, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m64_mae, tbg64_mae, b_idx=0)

In [ ]:
# Model 2 custom loss: m128_maew
tbg128_maew = model_complete_eval(m128_maew, h128_maew, pt128_maew, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_maew, tbg128_maew, b_idx=0)

In [ ]:
# Model 2 mae loss: m128_mae
tbg128_mae = model_complete_eval(m128_mae, h128_mae, pt128_mae, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_mae, tbg128_mae, b_idx=0)

In [ ]:
# Model 3 custom loss: m256_maew
tbg256_maew = model_complete_eval(m256_maew, h256_maew, pt256_maew, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m256_maew, tbg256_maew, b_idx=0)

In [ ]:
# Model 3 mae loss: m3_mae
tbg256_mae = model_complete_eval(m256_mae, h256_mae, pt256_mae, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m256_mae, tbg256_mae, b_idx=0)

## Test: Model/Net Topology
    Model 1:         128-128-1
    Model 1 + BN :   128-BN-128-BN-1
    Model 2:         128-128-64-1
    Model 3:         512-256-1
    Model 4:         32-16-1

In [ ]:
# Model 1: m128_tt
tbg128_tt = model_complete_eval(m128_tt, h128_tt, pt128_tt, params, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_tt, tbg128_tt, b_idx=0)

In [ ]:
# Model 1 + BN: m128bn_bs2048_tt
tbg128bn_bs2048_tt = model_complete_eval(m128bn_bs2048_tt, h128bn_bs2048_tt, pt128bn_bs2048_tt, params, batch_size=2048, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128bn_bs2048_tt, tbg128bn_bs2048_tt, b_idx=0)

In [ ]:
# Model 2: m128_128_64_tt
tbg128_128_64_tt = model_complete_eval(m128_128_64_tt, h128_128_64_tt, pt128_128_64_tt, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m128_128_64_tt, tbg128_128_64_tt, b_idx=0)

In [ ]:
# Model 3: m512_256_tt
tbg512_256_tt = model_complete_eval(m512_256_tt, h512_256_tt, pt512_256_tt, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
#pred, y = model_pred_on_gen_batch(m512_256_tt, tbg512_256_tt, b_idx=0)

In [ ]:
# Model 4: m32_16_tt not normalized?
tbg32_16_tt = model_complete_eval(m32_16_tt, h32_16_tt, pt32_16_tt, params_nsc, batch_size=1024, workers=NUM_WORKER)

#predict on model
pred, y = model_pred_on_gen_batch(m32_16_tt, tbg32_16_tt, b_idx=0)

### Results

In [ ]:
# Model 1: m128_tt
mae_p, mse_p, loc_p = model_eval_pos(m128_tt, h128_tt, part['NFCHOA_R006'], params, ID_ref, batch_size=1000, workers=NUM_WORKER)

## Error Comparison

In [6]:
part_x_M027, pos_ids = get_pos_IDs(part['NFCHOA_M027'],ID_ref)
part_x_R006, _ = get_pos_IDs(part['NFCHOA_R006'],ID_ref)

In [11]:
params_t = params.copy()
params_t['batch_size'] = 1000
#MSE, tau, sigma, _, _ = calc_errors_m(m32_16_tt, part_x, params_t, pos_ids, verbose=0, workers=NUM_WORKER)

In [12]:
n_model = len(m_flist)
MSE_R006 = np.zeros(n_model)
tau_sq_R006 = np.zeros(n_model)
sigma_sq_R006 = np.zeros(n_model)
MSE_M027 = np.zeros(n_model)
tau_sq_M027 = np.zeros(n_model)
sigma_sq_M027 = np.zeros(n_model)


for i in np.arange(n_model):
    model = load_model(model_dir + m_flist[i], custom_objects={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle})
    MSE_R006[i], tau_sq_R006[i], sigma_sq_R006[i], _, _ = calc_errors_m(model, part_x_R006, params_t, pos_ids, verbose=0, workers=NUM_WORKER)
    MSE_M027[i], tau_sq_M027[i], sigma_sq_M027[i], _, _ = calc_errors_m(model, part_x_M027, params_t, pos_ids, verbose=0, workers=NUM_WORKER)
    print(i)

In [43]:
m_nlist = []
loss_f = []
model_str = []
test_dset = []
bs = []
for i in np.arange(n_model):
    strn = m_flist[i].split('_')
    loss_f.append('{:<5}'.format(strn[1]))
    model_str.append('{:<15}'.format(strn[0]))
    test_dset.append('{:<15}'.format(strn[4]))
    bs.append('{:<5}'.format(strn[3]))
    m_nlist.append(strn[0]+'_'+strn[1]+'_'+strn[4])

data = {
    'model':model_str,
    'loss_f':loss_f,
    'test_dset':test_dset,
    'bs':bs,
    'MSE_R006' : MSE_R006,
    'sq_bias_R006': tau_sq_R006,
    'variance_R006': sigma_sq_R006,
    'MSE_M027': MSE_M027,
    'sq_bias_M027': tau_sq_M027,
    'variance_M027': sigma_sq_M027    
}
df = pd.DataFrame(data)

In [44]:
df

,model,loss_f,test_dset,bs,MSE_R006,sq_bias_R006,variance_R006,MSE_M027,sq_bias_M027,variance_M027
0,m032-16,maew,t-NR006-NM027,bs1024,4087.304869,708.878178,3378.426691,44.757406,1.459289,43.298117
1,m032-16,maew,t-NR006,bs1024,10699.216017,1.187886,10698.028131,10767.452257,3.353339,10764.098918
2,m064,mae,t-split,bs1024,1155.642512,68.268652,1087.373860,110.619235,0.857088,109.762147
3,m064,maew,t-split,bs1024,818.761054,55.538655,763.222399,33.777869,0.695579,33.082290
4,m128-128-64,maew,t-NR006,bs1024,10527.929683,14.551427,10513.378256,10996.390797,38.414172,10957.976624
5,m128,mae,t-split,bs1024,908.641999,92.604151,816.037848,65.856951,2.095613,63.761338
6,m128,maew,t-NHR006,bs1024,4754.408204,894.215146,3860.193058,22.239503,1.306412,20.933090
7,m128,maew,t-s2-20+onlyNHR300,bs1024,5744.493367,563.097252,5181.396114,1296.838450,345.233913,951.604537
8,m128,maew,t-s2-20,bs1024,1229.756666,210.448926,1019.307740,107.705314,4.670370,103.034944
9,m128,maew,t-split,bs1024,10767.369042,1.443750,10765.925292,10791.533164,1.082943,10790.450220


In [45]:
df[df['test_dset'].str.contains('R006')]

,model,loss_f,test_dset,bs,MSE_R006,sq_bias_R006,variance_R006,MSE_M027,sq_bias_M027,variance_M027
0,m032-16,maew,t-NR006-NM027,bs1024,4087.304869,708.878178,3378.426691,44.757406,1.459289,43.298117
1,m032-16,maew,t-NR006,bs1024,10699.216017,1.187886,10698.028131,10767.452257,3.353339,10764.098918
4,m128-128-64,maew,t-NR006,bs1024,10527.929683,14.551427,10513.378256,10996.390797,38.414172,10957.976624
6,m128,maew,t-NHR006,bs1024,4754.408204,894.215146,3860.193058,22.239503,1.306412,20.933090
11,m128bn-128bn,maew,t-NR006-NM027,bs256,4251.367588,589.897698,3661.469890,47.947149,1.393109,46.554040
12,m128bn,maew,t-NR006,bs128,4560.297875,502.311979,4057.985896,77.787690,0.896289,76.891401
13,m128bn,maew,t-NR006,bs2048,4967.464642,540.631296,4426.833346,152.881254,39.572754,113.308499
16,m512-256,maew,t-NR006,bs1024,10705.990517,6.226416,10699.764101,10815.129805,3.207599,10811.922206
